# ML models

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install Lime
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=eeb1cf9627750a5637df9b43d11aba80e77949008bd0feeaedb476519ae75613
  Stored in directory: /root/.cache/pip/wheels/e6/a6/20/cc1e293fcdb67ede666fed293cb895395e7ecceb4467779546
Successfully built lime


**Import Libraries**

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lime.lime_tabular

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE

In [4]:
# Load the dataset into a pandas DataFrame
df = pd.read_csv("/content/drive/MyDrive/MCS/MachineLearning/berka_dataset_with_state2.csv")

In [5]:
df.head()

,client_id,State,district_id,age,gender,district_of_residence,district_inhabitant_count,district_cities_count,district_urban_inhabitant_ratio,district_average_salary,...,cash_withdrawals,remittance_to_other_banks,has_had_loan,max_loan_amount,max_loan_duration,loan_pay_success,loan_pay_failure,in_loan_debt,year_of_last_transaction,year_of_last_credit
0,1.0,0,18.0,29.0,F,Pisek,70699.0,4.0,65.3,8968.0,...,97.0,40.0,False,0.0,0.0,False,False,False,98.0,98.0
1,2.0,1,1.0,54.0,M,Hl.m. Praha,1204953.0,1.0,100.0,12541.0,...,237.0,89.0,True,80952.0,24.0,True,False,False,98.0,98.0
2,3.0,1,1.0,59.0,F,Hl.m. Praha,1204953.0,1.0,100.0,12541.0,...,237.0,89.0,True,80952.0,24.0,True,False,False,98.0,98.0
3,4.0,0,5.0,43.0,M,Kolin,95616.0,6.0,51.4,9307.0,...,32.0,39.0,False,0.0,0.0,False,False,False,98.0,98.0
4,5.0,0,5.0,39.0,F,Kolin,95616.0,6.0,51.4,9307.0,...,32.0,39.0,False,0.0,0.0,False,False,False,98.0,98.0


In [6]:
# drop unnecessary columns, droping all the IDs
df = df.drop(columns=['client_id', 'account_id','district_id', 'disp_id', 'branch_location_id'], axis=1)  # Remove unnecessary columns

In [7]:
df.head()

,State,age,gender,district_of_residence,district_inhabitant_count,district_cities_count,district_urban_inhabitant_ratio,district_average_salary,district_mun_inhabitant_count_499,district_mun_inhabitant_count_1999,...,cash_withdrawals,remittance_to_other_banks,has_had_loan,max_loan_amount,max_loan_duration,loan_pay_success,loan_pay_failure,in_loan_debt,year_of_last_transaction,year_of_last_credit
0,0,29.0,F,Pisek,70699.0,4.0,65.3,8968.0,60.0,13.0,...,97.0,40.0,False,0.0,0.0,False,False,False,98.0,98.0
1,1,54.0,M,Hl.m. Praha,1204953.0,1.0,100.0,12541.0,0.0,0.0,...,237.0,89.0,True,80952.0,24.0,True,False,False,98.0,98.0
2,1,59.0,F,Hl.m. Praha,1204953.0,1.0,100.0,12541.0,0.0,0.0,...,237.0,89.0,True,80952.0,24.0,True,False,False,98.0,98.0
3,0,43.0,M,Kolin,95616.0,6.0,51.4,9307.0,65.0,30.0,...,32.0,39.0,False,0.0,0.0,False,False,False,98.0,98.0
4,0,39.0,F,Kolin,95616.0,6.0,51.4,9307.0,65.0,30.0,...,32.0,39.0,False,0.0,0.0,False,False,False,98.0,98.0


In [8]:
# Convert all categorical columns to label encoding

from sklearn.preprocessing import LabelEncoder

# create label encoder object
le = LabelEncoder()
  
df['gender']= le.fit_transform(df['gender'])
df['district_of_residence']= le.fit_transform(df['district_of_residence'])
df['owns_account']= le.fit_transform(df['owns_account'])
df['f_statement_issuance']= le.fit_transform(df['f_statement_issuance'])
df['branch_location']= le.fit_transform(df['branch_location'])
df['insurance_payment']= le.fit_transform(df['insurance_payment'])
df['household_payment']= le.fit_transform(df['household_payment'])
df['lease_payment']= le.fit_transform(df['lease_payment'])
df['loan_payment']= le.fit_transform(df['loan_payment'])
df['has_credit_card']= le.fit_transform(df['has_credit_card'])
df['has_had_loan']= le.fit_transform(df['has_had_loan'])
df['loan_pay_success']= le.fit_transform(df['loan_pay_success'])
df['loan_pay_failure']= le.fit_transform(df['loan_pay_failure'])
df['in_loan_debt']= le.fit_transform(df['in_loan_debt'])

In [9]:
df.head()

,State,age,gender,district_of_residence,district_inhabitant_count,district_cities_count,district_urban_inhabitant_ratio,district_average_salary,district_mun_inhabitant_count_499,district_mun_inhabitant_count_1999,...,cash_withdrawals,remittance_to_other_banks,has_had_loan,max_loan_amount,max_loan_duration,loan_pay_success,loan_pay_failure,in_loan_debt,year_of_last_transaction,year_of_last_credit
0,0,29.0,0,46,70699.0,4.0,65.3,8968.0,60.0,13.0,...,97.0,40.0,0,0.0,0.0,0,0,0,98.0,98.0
1,1,54.0,1,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,...,237.0,89.0,1,80952.0,24.0,1,0,0,98.0,98.0
2,1,59.0,0,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,...,237.0,89.0,1,80952.0,24.0,1,0,0,98.0,98.0
3,0,43.0,1,29,95616.0,6.0,51.4,9307.0,65.0,30.0,...,32.0,39.0,0,0.0,0.0,0,0,0,98.0,98.0
4,0,39.0,0,29,95616.0,6.0,51.4,9307.0,65.0,30.0,...,32.0,39.0,0,0.0,0.0,0,0,0,98.0,98.0


In [10]:
df.columns

Index(['State', 'age', 'gender', 'district_of_residence',
       'district_inhabitant_count', 'district_cities_count',
       'district_urban_inhabitant_ratio', 'district_average_salary',
       'district_mun_inhabitant_count_499',
       'district_mun_inhabitant_count_1999',
       'district_mun_inhabitant_count_9999',
       'district_mun_inhabitant_count_infinite',
       'district_enterprenuer_per_1000', 'district_crimes_count',
       'district_unemployment_rate', 'owns_account', 'tenure',
       'f_statement_issuance', 'branch_location', 'orders_count',
       'insurance_payment', 'household_payment', 'lease_payment',
       'loan_payment', 'has_credit_card', 'transaction_count',
       'withdraw_transactions', 'credit_transactions',
       'credit_card_withdrawals', 'credit_in_cash',
       'collections_from_other_banks', 'cash_withdrawals',
       'remittance_to_other_banks', 'has_had_loan', 'max_loan_amount',
       'max_loan_duration', 'loan_pay_success', 'loan_pay_failure',


In [11]:
#scale all features except our target feature
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()  # scale the features
features = ['age', 'gender', 'district_of_residence',
       'district_inhabitant_count', 'district_cities_count',
       'district_urban_inhabitant_ratio', 'district_average_salary',
       'district_mun_inhabitant_count_499',
       'district_mun_inhabitant_count_1999',
       'district_mun_inhabitant_count_9999',
       'district_mun_inhabitant_count_infinite',
       'district_enterprenuer_per_1000', 'district_crimes_count',
       'district_unemployment_rate', 'owns_account', 'tenure',
       'f_statement_issuance', 'branch_location', 'orders_count',
       'insurance_payment', 'household_payment', 'lease_payment',
       'loan_payment', 'has_credit_card', 'transaction_count',
       'withdraw_transactions', 'credit_transactions',
       'credit_card_withdrawals', 'credit_in_cash',
       'collections_from_other_banks', 'cash_withdrawals',
       'remittance_to_other_banks', 'has_had_loan', 'max_loan_amount',
       'max_loan_duration', 'loan_pay_success', 'loan_pay_failure',
       'in_loan_debt', 'year_of_last_transaction', 'year_of_last_credit']

In [12]:
df.head()

,State,age,gender,district_of_residence,district_inhabitant_count,district_cities_count,district_urban_inhabitant_ratio,district_average_salary,district_mun_inhabitant_count_499,district_mun_inhabitant_count_1999,...,cash_withdrawals,remittance_to_other_banks,has_had_loan,max_loan_amount,max_loan_duration,loan_pay_success,loan_pay_failure,in_loan_debt,year_of_last_transaction,year_of_last_credit
0,0,29.0,0,46,70699.0,4.0,65.3,8968.0,60.0,13.0,...,97.0,40.0,0,0.0,0.0,0,0,0,98.0,98.0
1,1,54.0,1,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,...,237.0,89.0,1,80952.0,24.0,1,0,0,98.0,98.0
2,1,59.0,0,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,...,237.0,89.0,1,80952.0,24.0,1,0,0,98.0,98.0
3,0,43.0,1,29,95616.0,6.0,51.4,9307.0,65.0,30.0,...,32.0,39.0,0,0.0,0.0,0,0,0,98.0,98.0
4,0,39.0,0,29,95616.0,6.0,51.4,9307.0,65.0,30.0,...,32.0,39.0,0,0.0,0.0,0,0,0,98.0,98.0


# Classification

In [13]:
# Divide the data into features and labels
X = df.drop(["State"], axis=1)
y = df["State"]

In [14]:
X

,age,gender,district_of_residence,district_inhabitant_count,district_cities_count,district_urban_inhabitant_ratio,district_average_salary,district_mun_inhabitant_count_499,district_mun_inhabitant_count_1999,district_mun_inhabitant_count_9999,...,cash_withdrawals,remittance_to_other_banks,has_had_loan,max_loan_amount,max_loan_duration,loan_pay_success,loan_pay_failure,in_loan_debt,year_of_last_transaction,year_of_last_credit
0,29.0,0,46,70699.0,4.0,65.3,8968.0,60.0,13.0,2.0,...,97.0,40.0,0,0.0,0.0,0,0,0,98.0,98.0
1,54.0,1,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,0.0,...,237.0,89.0,1,80952.0,24.0,1,0,0,98.0,98.0
2,59.0,0,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,0.0,...,237.0,89.0,1,80952.0,24.0,1,0,0,98.0,98.0
3,43.0,1,29,95616.0,6.0,51.4,9307.0,65.0,30.0,4.0,...,32.0,39.0,0,0.0,0.0,0,0,0,98.0,98.0
4,39.0,0,29,95616.0,6.0,51.4,9307.0,65.0,30.0,4.0,...,32.0,39.0,0,0.0,0.0,0,0,0,98.0,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5364,54.0,0,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,0.0,...,136.0,77.0,1,419880.0,60.0,0,0,0,98.0,98.0
5365,56.0,1,17,1204953.0,1.0,100.0,12541.0,0.0,0.0,0.0,...,136.0,77.0,1,419880.0,60.0,0,0,0,98.0,98.0
5366,31.0,1,67,117897.0,6.0,53.8,8814.0,139.0,28.0,5.0,...,148.0,104.0,1,54024.0,12.0,1,0,0,98.0,98.0
5367,37.0,0,6,106054.0,6.0,63.1,8110.0,38.0,25.0,6.0,...,103.0,160.0,1,129408.0,24.0,1,0,0,98.0,98.0


In [15]:
X = StandardScaler().fit_transform(X) # scale the features

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Random Forest Classifier**

In [17]:
# Initialize a random forest classifier
clf = RandomForestClassifier(n_estimators=100)

In [18]:
# Train the classifier on the training data
clf.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
# Make predictions on the test data
y_pred = clf.predict(X_test)

In [22]:
# Print the accuracy of the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9925512104283054


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       724
           1       0.99      0.98      0.99       350

    accuracy                           0.99      1074
   macro avg       0.99      0.99      0.99      1074
weighted avg       0.99      0.99      0.99      1074



**AdaBoost Classifier**

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# Initialize a AdaBoostClassifier
adaBoost = AdaBoostClassifier()

In [ ]:
# Train the classifier on the training data
adaBoost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
# Make predictions on the test data
y_pred = adaBoost.predict(X_test)

In [ ]:
# Print the accuracy of the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9934823091247672


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       724
           1       0.99      0.99      0.99       350

    accuracy                           0.99      1074
   macro avg       0.99      0.99      0.99      1074
weighted avg       0.99      0.99      0.99      1074



**Logistic Regrassion**

In [ ]:
# Initialize a Logistic Regression
lr = LogisticRegression()
# Train
lr.fit(X_train, y_train)
# Make prediction
y_pred = lr.predict(X_test)
# print the report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       724
           1       0.98      0.98      0.98       350

    accuracy                           0.99      1074
   macro avg       0.99      0.99      0.99      1074
weighted avg       0.99      0.99      0.99      1074



In [ ]:
# skf = StratifiedKFold(n_splits=3)
_scoring = ["accuracy", "precision", "recall", "f1"]
results = cross_validate(estimator=AdaBoostClassifier(),
                               X=X_test,
                               y=y_test,
                               cv=5,
                               scoring=_scoring,
                               return_train_score=True)
      
{
    "Training Accuracy scores": results['train_accuracy'],
    "Mean Training Accuracy": results['train_accuracy'].mean()*100,
    "Training Precision scores": results['train_precision'],
    "Mean Training Precision": results['train_precision'].mean(),
    "Training Recall scores": results['train_recall'],
    "Mean Training Recall": results['train_recall'].mean(),
    "Training F1 scores": results['train_f1'],
    "Mean Training F1 Score": results['train_f1'].mean(),
    "Validation Accuracy scores": results['test_accuracy'],
    "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
    "Validation Precision scores": results['test_precision'],
    "Mean Validation Precision": results['test_precision'].mean(),
    "Validation Recall scores": results['test_recall'],
    "Mean Validation Recall": results['test_recall'].mean(),
    "Validation F1 scores": results['test_f1'],
    "Mean Validation F1 Score": results['test_f1'].mean()
}

{'Training Accuracy scores': array([1., 1., 1., 1., 1.]),
 'Mean Training Accuracy': 100.0,
 'Training Precision scores': array([1., 1., 1., 1., 1.]),
 'Mean Training Precision': 1.0,
 'Training Recall scores': array([1., 1., 1., 1., 1.]),
 'Mean Training Recall': 1.0,
 'Training F1 scores': array([1., 1., 1., 1., 1.]),
 'Mean Training F1 Score': 1.0,
 'Validation Accuracy scores': array([0.98139535, 0.98604651, 0.98139535, 0.97674419, 0.97663551]),
 'Mean Validation Accuracy': 98.04433818735058,
 'Validation Precision scores': array([0.97142857, 0.98550725, 0.98529412, 0.94520548, 0.95774648]),
 'Mean Validation Precision': 0.9690363787555472,
 'Validation Recall scores': array([0.97142857, 0.97142857, 0.95714286, 0.98571429, 0.97142857]),
 'Mean Validation Recall': 0.9714285714285713,
 'Validation F1 scores': array([0.97142857, 0.97841727, 0.97101449, 0.96503497, 0.96453901]),
 'Mean Validation F1 Score': 0.9700868604992816}